# De novo assembly of Illumina reads

**1)** Connect to the cluster and then connect to the software environment.

In [ ]:
%%bash
module load Anaconda3/5.1.0
source activate forbio_env


________

**2)** Copy the folder for the assembly tutorial to your account and enter the folder:

In [ ]:
%%bash
cp -r /work/projects/forbio/tutorials_tobi/de_novo_assembly .
cd de_novo_assembly


_______

**3)** Prepare and run the de novo assembly job. This is what the basic syntax of a very simple abyss assembly command looks like (for more information see the [abyss manual](https://docs.google.com/document/d/1WUy2VYYWEfcPm3_PldXu8z_VVxCoQvfmIoL7ysucx-c/edit)): 
```
abyss-pe \
    --directory=OUTPUT_DIR \
    k=KMER_VALUE \
    name=SAMPLE_ID \
    in="/PATH/TO/MY_READS_R1.fastq /PATH/TO/MY_READS_R2.fastq"
```

Run this command in the command line (after inserting the correct values and file-paths) to test if you get it to work. If you get it to run successfully (no error messages), **STOP THE COMMAND** by pressing `Ctrl+D`. We are cancelling it for the reason that the command needs to run for a while (ca. 30 min) and we want to submit it as a jobscript to the queue-system with `sbatch` instead (instructions below).

<div class="alert alert-block alert-info">
INFO: You don't need to type the ` \`, which is only needed to continue your command over several lines in the command line. Instead you can just put everything in one line.
</div>

Knowing the abyss command syntax from above, **complete the abyss command** in the job-script `assembly_abyss.sh` in your tutorial folder. 

<div class="alert alert-block alert-info">
INFO: You only need to edit the lines commented with 'CHANGE' in the document.
</div>

You find the fastq files to assemble in the `de_novo_assembly` tutorial directory in the folder `fastq_reads`.

When the file is set up, submit the job using `sbatch`. Check if the job is running with `squeue -u YOUR_USERNAME`.

<div class="alert alert-block alert-info">
INFO: The job will run in the background for ca. 30 min. In the meantime move on to the next step.
</div>


In [ ]:
%%bash
# do the changes in the file by opening it in text editor
nano assembly_abyss.sh
# submit the job
sbatch assembly_abyss.sh
# check if the job is running
squeue -u YOUR_ABEL_USER_ID


_____

**4)** Try out different kmer values. Change the kmer value in the job script and submit again. Try at least 3 different values between 20 and 80.

<div class="alert alert-block alert-success">
If all assemblies (different kmer values) are running properly, you have ca. 30 min of time until all your assemblies are finished. Go grab a coffee and/or seize the time to have a look at some of the publications in the [course literature folder](https://drive.google.com/drive/u/0/folders/1VjMMVMf3XWi_Vm7-r6C6_GXuFwSzpk_z).

Not only because of shameless self-advertizing, but because of the description of the workflow relevant for this course, I recommend you to have a look at [this article](https://peerj.com/articles/5175/)


</div>



______

**5)** Evaluate assembly results. There are a lot of files in the assembly output. The file containing the final contig sequences is stored in the assembly output folder under the name `T_pella5-contigs.fa`. Check it out using the following commands:

In [ ]:
%%bash
# print first lines of file to screen
head -n 10 T_pella5-contigs.fa
# print last lines of file to screen
tail -n 10 T_pella5-contigs.fa


Try to figure out what information is stored in the sequence headers. Can you find out **how many contigs** were produced? And what about the **length of the contig sequences**? What differences can you find between the different abyss runs with different kmer values?

I wrote a small program (`plot_contig_length.sh`) that plots the distribution of sequence lengths to the screen. The program is found in the `scripts` folder and can be executed like this (the script will take a minute or so to finish):

In [ ]:
%%bash
sh ./plot_contig_length.sh ./path/to/T_pella5-contigs.fa 


_____________________

### Blast contig against NCBI GeneBank:
Extract the longest contig from the contig file using the program `fp.py`, which you can install with the command below:

In [ ]:
%%bash
wget https://raw.githubusercontent.com/mtop/ngs/master/fp.py

You execute this program like this:

In [ ]:
%%bash
python2.7 /PATH/TO/fp.py -h

Use `fp.py` to extract the longest contig sequence from the contig file. Extract it and write it to a separate fasta file `longest_contig.fasta`:

In [ ]:
%%bash
python2.7 /PATH/TO/fp.py --longest ./path/to/contigs/T_pella5-contigs.fa > longest_contig.fasta

Then run a blast search against NCBI GeneBank to see what this sequence represents. You can use the blast commandline tool to run a blast search against the online database:

In [ ]:
%%bash
module load blast+
blastx -query ./longest_contig.fasta -db nr -out ./blast_results.txt -remote

I prepared a job-script, which you find at `scripts/run_blast_search.sh`. Execute the script from the folder containing the file `longest_contig.fasta`. The search will take at least 10 minutes.


______

### Blast contigs against mitochondrial genome:
Let's say you don't want to search through all of GeneBank but instead want to test if you have contigs representing a certain sequence, say the mitochondrial genome. In that case we can find a mitochondrial genome sequence of a close ancestor on GeneBank, download it and build our own blast reference with it.

Since we are using hummingbird data, let's see if we can find any contigs of mitochondrial origin by blasting the contigs against the mitochondrial genome of a related hummingbird. I stored a reference sequence of the hummingbird *Amazilla versicolor* in the subfolder `reference_sequences`. You first have to create a blast database with that sequence using the following command:

In [ ]:
%%bash
makeblastdb -dbtype nucl -in reference_sequences/amazilla_versicolor_mitochondrion.fasta

Now you can run a blast search against this local database using all contigs that were assembled:

In [ ]:
%%bash
blastn -query ./path/to/T_pella5-contigs.fa -db reference_sequences/amazilla_versicolor_mitochondrion.fasta -out blast_mitoch_results.txt -outfmt 6

Inspect the output file, which contains a list of all contigs that could be matched to the mitochondrial genome. The field-codes of the columns in the output file:
`query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score`



______

### Blast contigs against bait set:

**1) Generate UCE fasta library.** The data we are working with were generated using an Ultraconserved Elements probe kit (one probe per target locus). Let's now see how many and which contigs match with the UCE probe sequences. Create a local blast database for the file `reference_sequences/Tetrapods-UCE-2.5Kv1.fasta` (see above).

**2) Run blast search.** Now blast the contigs file against the UCE reference database, using the command from above.

**3) Evaluate results.** Look at the output file from the blast command. What can you say about the matches that were found? How similar are they to the references in the reference library (= probe sequences)? Do you have any comments on the number of hits which we retained (check the number of hits with the command below)?

In [ ]:
%%bash
cat blast_uce_results.txt | wc -l